<h1 style='color:blue;'>Accès à une BD Cassandra avec Python</h1>

---

**Auteur :** Dr. Hmida HMIDA

# Installer le driver cassandra

In [ ]:
!pip3 install cassandra-driver

# Connexion au serveur

In [ ]:
from cassandra.cluster import Cluster
#cluster = Cluster()
# ou en spécifiant @IP et le port
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()

## Connexion avec mot de passe

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider = PlainTextAuthProvider(username='user', password='password')
cluster = Cluster(auth_provider=auth_provider, protocol_version=2)
session2 = cluster.connect()

# Choix du keyspace

In [ ]:
session.set_keyspace('resto_ny')
## ou 
# session.execute('use resto_ny')

# Exécuter des requêtes

In [ ]:
rows = session.execute('SELECT name, borough, zipcode FROM restaurant limit 10')
header = 'Name'.ljust(70)+'|Borough'.ljust(30)+'|Zipcode'.ljust(10)
print('-'*len(header))
print(header)
print('-'*len(header))
for resto in rows:
    print (resto.name.ljust(70)+'|'+resto.borough.ljust(30)+'|'+str(resto.zipcode).ljust(10))
    # ou par position
    # print (resto[0].ljust(70)+'|'+resto[1].ljust(30)+'|'+str(resto[2]).ljust(10))
print('-'*len(header))

# Prepared Statement
**Essayer avec les valeurs 10023, 10033, ...**

In [ ]:
rechParZipcode = session.prepare('SELECT name, borough, zipcode FROM restaurant WHERE zipcode = ? LIMIT 10 ALLOW FILTERING')
zcode = int(input('Entrer le code postal :'))
rows = session.execute(rechParZipcode,[zcode])
header = 'Name'.ljust(70)+'|Borough'.ljust(30)+'|Zipcode'.ljust(10)
print('-'*len(header))
print(header)
print('-'*len(header))
for resto in rows:
    print (resto.name.ljust(70)+'|'+resto.borough.ljust(30)+'|'+str(resto.zipcode).ljust(10))
    # ou par position
    # print (resto[0].ljust(70)+'|'+resto[1].ljust(30)+'|'+str(resto[2]).ljust(10))
print('-'*len(header))

# Fermeture de la session

In [ ]:
session.shutdown()

# Utilisation d'un UDT

In [ ]:
from cassandra.cluster import Cluster
cluster = Cluster(protocol_version=3)
session = cluster.connect()
session.set_keyspace('resto_ny')
session.execute("CREATE TYPE IF NOT EXISTS address (street text, zipcode int)")
session.execute("CREATE TABLE IF NOT EXISTS users (id int PRIMARY KEY, location frozen<address>)")

cluster.register_user_type('resto_ny', 'address', dict)

# insert a row using a prepared statement and a tuple
insert_statement = session.prepare("INSERT INTO users (id, location) VALUES (?, ?)")
session.execute(insert_statement, [0, ("123 Main St.", 78723)])

# results will include dict instances
results = session.execute("SELECT * FROM users")
row = results.one()
print(row.id, row.location['street'], row.location['zipcode'])

---
<h1 style="color:green"> Exercice 1</h1>

Écrire un programme Python permettant de :

- Saisir un quartier (borough)
- Chercher les restaurants de ce quartier
- Afficher pour chaque restaurant trouvé les dates des inspections pour lesquelles il a reçu un grade 'B' 

In [ ]:
# Votre code ici

---
<h1 style="color:green"> Exercice 2</h1>

Écrire un programme Python qui  importe un fichier JSON dans une table Cassandra. 

In [ ]:
# Votre code ici

---
<h1 style="color:green"> Exercice 3</h1>

Écrire un programme Python qui fusionne les tables ***Restaurant*** et ***Inspection*** en une seule table RestoInspectionV2 avec intégrant les inspections comme un map dans les restaurants (Second Besoin dans le TP3) 

In [ ]:
# Votre code ici